In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import pprint
%matplotlib inline

import os
print(os.listdir("../input"))

['Colo - Copy (2).csv', 'Colo.csv', 'my_model', 'my_model.h5', 'Preciption_(2020)', 'Preciption_(2020).h5', 'Precption(2).csv', 'Precption(3).csv', 'Precption(4).csv', 'Precption.csv']


In [29]:
# Recreate the exact same model, including its weights and the optimizer
regressor = tf.keras.models.load_model('../input/Preciption_(2020)')

# Show the model architecture
regressor.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 60, 150)           91200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 150)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 60, 150)           180600    
_________________________________________________________________
dropout_2 (Dropout)          (None, 60, 150)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 60, 150)           180600    
_________________________________________________________________
dropout_3 (Dropout)          (None, 60, 150)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 150)              

In [30]:
df = pd.read_csv('../input/Colo - Copy (2).csv')

In [31]:
df.Date=pd.to_datetime(df.Date)
df=df.set_index('Date')

In [32]:
TestData = df.tail(365)

In [33]:
Df_Total = pd.concat((df[["Precip MM"]], TestData[["Precip MM"]]), axis=0)

In [34]:
Df_Total.shape

(4382, 1)

In [35]:
inputs = Df_Total[len(Df_Total) - len(TestData) - 60:].values
inputs.shape

(425, 1)

In [36]:
from pandas.tseries.offsets import DateOffset
add_dates = [df.index[-1] + DateOffset(days=x) for x in range(0,366)]
future_dates = pd.DataFrame(index=add_dates[1:],columns=df.columns)

In [37]:
future_dates.tail(365)

,Precip MM
2020-01-01,NaN
2020-01-02,NaN
2020-01-03,NaN
2020-01-04,NaN
2020-01-05,NaN
...,...
2020-12-26,NaN
2020-12-27,NaN
2020-12-28,NaN
2020-12-29,NaN


In [38]:
future_dates.shape

(365, 1)

In [39]:
inputs = Df_Total[len(Df_Total) - len(TestData) - 60:].values

# We need to Reshape
inputs = inputs.reshape(-1,1)

# Normalize the Dataset
inputs = sc.transform(inputs)

X_test = []
for i in range(60, 425):
    X_test.append(inputs[i-60:i])
       
# Convert into Numpy Array
X_test = np.array(X_test)

# Reshape before Passing to Network
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Pass to Model
predicted_Precip = regressor.predict(X_test)

# Do inverse Transformation to get Values
predicted_Precip = sc.inverse_transform(predicted_Precip)

In [40]:
True_Precip = 0
Predicted_Precip  = predicted_Precip
dates = TestData.index.to_list()

In [43]:
#df_predict = pd.DataFrame(scaler.inverse_transform(pred_list), index=future_dates[-n_input:].index, columns=['Predictions'])
Machine_Df2 = pd.DataFrame(data={
    "Date":future_dates.index.to_list(),
    "TruePrecip": True_Precip,
    "PredictedPrecip":[x[0] for x in predicted_Precip]
})

In [44]:
Machine_Df2

,Date,TruePrecip,PredictedPrecip
0,2020-01-01,0,4.044505
1,2020-01-02,0,1.671005
2,2020-01-03,0,1.965037
3,2020-01-04,0,1.572994
4,2020-01-05,0,1.357126
...,...,...,...
360,2020-12-26,0,2.586493
361,2020-12-27,0,1.362504
362,2020-12-28,0,1.741016
363,2020-12-29,0,1.877571


In [ ]:
df_proj.to_csv("../input/Precption(4).csv")